In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from scipy.stats import probplot, boxcox
from scipy.special import inv_boxcox
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
test = pd.read_csv("test.csv",index_col='row_id')
train = pd.read_csv("train.csv",index_col='row_id')
sub = pd.read_csv("sample_submission.csv",index_col='row_id')

In [28]:
def add_p_t(train):
    dfc = train.groupby(['product','date']).sum()/train.groupby(['date']).sum()

    df = train.copy()

    dfc = dfc.unstack(0)
    dfc.columns = dfc.columns.droplevel()
    dfc = dfc.reset_index()
    dfc = dfc.melt(id_vars = 'date')
    dfc.columns = ['date','product','P']
    train = df.reset_index()
    train = pd.merge(left=train,right=dfc,on=['date','product'],how='left')


    train['T'] = train['num_sold']/train['P']
    return train

test = add_p_t(test)


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [21]:
train

,row_id,date,country,store,product,num_sold,P,T
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,0.277944,2385.370518
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615,0.227462,2703.745704
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480,0.188900,2541.020690
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,0.305693,2322.591093
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240,0.277944,863.482540
...,...,...,...,...,...,...,...,...
70123,70123,2020-12-31,Spain,KaggleMart,Kaggle for Kids: One Smart Goose,614,0.292525,2098.963067
70124,70124,2020-12-31,Spain,KaggleRama,Kaggle Advanced Techniques,215,0.291703,737.050121
70125,70125,2020-12-31,Spain,KaggleRama,Kaggle Getting Started,158,0.221948,711.877778
70126,70126,2020-12-31,Spain,KaggleRama,Kaggle Recipe Book,135,0.193823,696.511663


In [22]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

train['Year'] = train['date'].dt.year
train['Month'] = train['date'].dt.month
train['Day'] = train['date'].dt.day

test['Year'] = test['date'].dt.year
test['Month'] = test['date'].dt.month
test['Day'] = test['date'].dt.day
        
def by_10(x):
    if x.Day<10:
        x.Month = 1000*int(x.Month)
    else:
        x.Month = 100 * int(x.Month)
    return x

train['weekday'] = train['date'].dt.weekday
train['amount_time'] = train['Month']*100 + train['Day']



test['weekday'] = test['date'].dt.weekday
test['amount_time'] = test['Month']*100 + test['Day']


special_days = [101,1228,1229,1230,1231]
for day in special_days:
    title = "special_day_" + str(day)
    train[title] = train['amount_time'].isin([day]).astype(int)
    test[title] = test['amount_time'].isin(special_days).astype(int)
    


    
train['Month'] = np.cos(0.5236 * train['Month'])
test["Month"] = np.cos(0.5236 * test['Month'])

test.drop("date", axis = 1,inplace = True)
train.drop("date" , axis = 1,inplace = True)


In [116]:
train

,country,store,product,num_sold,Year,Month,Day,weekday,amount_time,special_day_101,special_day_1228,special_day_1229,special_day_1230,special_day_1231
row_id,,,,,,,,,,,,,,
0,Belgium,KaggleMart,Kaggle Advanced Techniques,663,2017,0.866025,1,6,101,1,0,0,0,0
1,Belgium,KaggleMart,Kaggle Getting Started,615,2017,0.866025,1,6,101,1,0,0,0,0
2,Belgium,KaggleMart,Kaggle Recipe Book,480,2017,0.866025,1,6,101,1,0,0,0,0
3,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,2017,0.866025,1,6,101,1,0,0,0,0
4,Belgium,KaggleRama,Kaggle Advanced Techniques,240,2017,0.866025,1,6,101,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70123,Spain,KaggleMart,Kaggle for Kids: One Smart Goose,614,2020,1.000000,31,3,1231,0,0,0,0,1
70124,Spain,KaggleRama,Kaggle Advanced Techniques,215,2020,1.000000,31,3,1231,0,0,0,0,1
70125,Spain,KaggleRama,Kaggle Getting Started,158,2020,1.000000,31,3,1231,0,0,0,0,1


In [49]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)


In [50]:
test

,Year,Month,Day,weekday,amount_time,special_day_101,special_day_1228,special_day_1229,special_day_1230,special_day_1231,...,country_Germany,country_Italy,country_Poland,country_Spain,store_KaggleMart,store_KaggleRama,product_Kaggle Advanced Techniques,product_Kaggle Getting Started,product_Kaggle Recipe Book,product_Kaggle for Kids: One Smart Goose
row_id,,,,,,,,,,,,,,,,,,,,,
70128,2021,0.866025,1,4,101,1,1,1,1,1,...,0,0,0,0,1,0,1,0,0,0
70129,2021,0.866025,1,4,101,1,1,1,1,1,...,0,0,0,0,1,0,0,1,0,0
70130,2021,0.866025,1,4,101,1,1,1,1,1,...,0,0,0,0,1,0,0,0,1,0
70131,2021,0.866025,1,4,101,1,1,1,1,1,...,0,0,0,0,1,0,0,0,0,1
70132,2021,0.866025,1,4,101,1,1,1,1,1,...,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87643,2021,1.000000,31,4,1231,1,1,1,1,1,...,0,0,0,1,1,0,0,0,0,1
87644,2021,1.000000,31,4,1231,1,1,1,1,1,...,0,0,0,1,0,1,1,0,0,0
87645,2021,1.000000,31,4,1231,1,1,1,1,1,...,0,0,0,1,0,1,0,1,0,0


In [51]:
X = train.drop('num_sold', axis=1).values
y = train['num_sold']
X_test = test.values


print(y.shape)
bc_result = boxcox(y) #normalisation
boxcox_y = bc_result[0]
lam = bc_result[1] #lambda choisi par l'aglgo

sc = StandardScaler() #standardisation
#X = sc.fit_transform(X)
#X_test = sc.fit_transform(X_test)

(70128,)


In [52]:
model = xgb.XGBRegressor()

model.fit(X, boxcox_y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [53]:
clf = CatBoostRegressor()
clf.fit(X, boxcox_y)

Learning rate set to 0.080137
0:	learn: 1.3283535	total: 6.81ms	remaining: 6.8s
1:	learn: 1.2323336	total: 13.9ms	remaining: 6.92s
2:	learn: 1.1458094	total: 19.5ms	remaining: 6.47s
3:	learn: 1.0674933	total: 25.4ms	remaining: 6.32s
4:	learn: 0.9944132	total: 31.1ms	remaining: 6.18s
5:	learn: 0.9291850	total: 36.7ms	remaining: 6.08s
6:	learn: 0.8696338	total: 42.4ms	remaining: 6.01s
7:	learn: 0.8142340	total: 48ms	remaining: 5.95s
8:	learn: 0.7643035	total: 53.7ms	remaining: 5.91s
9:	learn: 0.7195748	total: 59.5ms	remaining: 5.89s
10:	learn: 0.6780253	total: 65ms	remaining: 5.84s
11:	learn: 0.6406521	total: 71.8ms	remaining: 5.91s
12:	learn: 0.6070377	total: 78.1ms	remaining: 5.93s
13:	learn: 0.5757599	total: 84.1ms	remaining: 5.92s
14:	learn: 0.5486410	total: 90.4ms	remaining: 5.94s
15:	learn: 0.5239495	total: 95.9ms	remaining: 5.9s
16:	learn: 0.4990208	total: 102ms	remaining: 5.88s
17:	learn: 0.4763951	total: 108ms	remaining: 5.87s
18:	learn: 0.4575074	total: 113ms	remaining: 5.85s
1

In [54]:
result = inv_boxcox(model.predict(X_test), lam)

result2 = inv_boxcox(clf.predict(X_test), lam)


In [55]:
sub["num_sold"] = 0.5*result2+0.5*result

In [56]:
sub.to_csv("result.csv")